In [7]:
import numpy as np

In [8]:
def user_similarity(matrix, u_a, u_b):
    col_a = matrix[:, u_a]
    col_b = matrix[:, u_b]

    mask = (col_a != 0) & (col_b != 0)
    if mask.sum() == 0:
        return 0.0

    vec_a = col_a[mask]
    vec_b = col_b[mask]

    top = np.dot(vec_a, vec_b)
    bottom = np.linalg.norm(vec_a) * np.linalg.norm(vec_b)

    return top / bottom if bottom != 0 else 0.0


# Формирование матрицы парных сходств пользователей
def compute_user_sim_matrix(matrix):
    users_count = matrix.shape[1]
    sim = np.zeros((users_count, users_count))

    for u in range(users_count):
        for v in range(u + 1, users_count):
            s = user_similarity(matrix, u, v)
            sim[u, v] = s
            sim[v, u] = s
    return sim

In [9]:
# Средний рейтинг пользователя (без нулей)
def mean_rating(matrix, user):
    ratings = matrix[:, user]
    non_zero = ratings[ratings != 0]
    return non_zero.mean() if non_zero.size > 0 else 0.0


# Прогноз оценки товара пользователем на основе соседей
def forecast_score(matrix, sim, user, product, neighbors=2):
    available = [u for u in range(matrix.shape[1]) if u != user and matrix[product, u] != 0]
    if len(available) == 0:
        return None, []

    similarities = np.array([sim[user, u] for u in available])
    chosen = np.argsort(similarities)[-neighbors:]
    chosen_users = [available[i] for i in chosen]

    base = mean_rating(matrix, user)
    numerator, denominator = 0.0, 0.0

    for other in chosen_users:
        numerator += sim[user, other] * (matrix[product, other] - mean_rating(matrix, other))
        denominator += abs(sim[user, other])

    if denominator == 0:
        return None, chosen_users

    predicted = base + numerator / denominator
    return predicted, chosen_users


# Рекомендации по популярности (для новых или холодных пользователей)
def top_popular(matrix):
    avg_scores = np.array([
        row[row != 0].mean() if np.any(row != 0) else 0
        for row in matrix
    ])
    return avg_scores.argmax(), avg_scores

In [12]:
def advise(matrix, sim, user, product=None, neighbors=2, threshold=4.0, user_labels=None, item_labels=None):
    uname = user_labels[user] if user_labels else f"user{user+1}"

    # если у пользователя нет данных — используем популярность
    if np.all(matrix[:, user] == 0):
        p_index, avg = top_popular(matrix)
        pname = item_labels[p_index] if item_labels else f"item{p_index+1}"
        print(f"\n{uname} — новый пользователь - советуем самый популярный товар: {pname} (средн. рейтинг {avg[p_index]:.2f})")
        return

    pname = item_labels[product] if item_labels else f"item{product+1}"
    prediction, neighbors_list = forecast_score(matrix, sim, user, product, neighbors)

    print(f"\nОценка для {uname} по товару {pname}:")
    print("Соседи:", [user_labels[u] for u in neighbors_list] if user_labels else neighbors_list)

    if prediction is None:
        print("Недостаточно данных для прогноза")
        return

    print(f"Прогнозируемый балл: {prediction:.3f}")
    avg_user = mean_rating(matrix, user)

    if prediction >= threshold and prediction >= avg_user:
        print(f"Рекомендуем: {prediction:.2f} ≥ {threshold} и ≥ средн. {avg_user:.2f}")
    else:
        print(f"Не рекомендуем: {prediction:.2f} < {threshold} или < средн. {avg_user:.2f}")

In [13]:
matrix = np.array([
    [4, 0, 5, 3, 0],
    [0, 3, 4, 0, 5],
    [5, 4, 0, 4, 3],
    [3, 5, 2, 0, 4],
    [0, 4, 0, 5, 5],
])
users = ["U1", "U2", "U3", "U4", "U5"]
items = ["P1", "P2", "P3", "P4", "P5"]

sim_matrix = compute_user_sim_matrix(matrix)

advise(matrix, sim_matrix, 0, 4, user_labels=users, item_labels=items)
advise(matrix, sim_matrix, 1, 3, user_labels=users, item_labels=items)

# Добавление нового пользователя
matrix_new = np.column_stack([matrix, np.zeros(matrix.shape[0])])
users_new = users + ["U6"]
advise(matrix_new, None, 5, user_labels=users_new, item_labels=items)


Оценка для U1 по товару P5:
Соседи: ['U2', 'U4']
Прогнозируемый балл: 4.516
Рекомендуем: 4.52 ≥ 4.0 и ≥ средн. 4.00

Оценка для U2 по товару P4:
Соседи: ['U1', 'U5']
Прогнозируемый балл: 3.378
Не рекомендуем: 3.38 < 4.0 или < средн. 4.00

U6 — новый пользователь - советуем самый популярный товар: P5 (средн. рейтинг 4.67)
